In [1]:
from docx import Document
from pandas import DataFrame, concat
import pandas as pd
import re

In [2]:
def read_docx(file_path):
    doc = Document(file_path)
    return doc

In [3]:
def read_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return "\n".join(full_text)


In [4]:
file_path = "../data/subject1.docx"
text = read_docx(file_path)
print(text)

DIREITO
HOMEM
ÚLTIMO ANO
CIDADE OUTRA
Entrevistadora: Ele, ele pega, pode ficar normal, tá? Ele pega o som bem bom. E pra começar eu queria saber o seu nome completo, a sua idade e o período que cê tá do curso? 
Entrevistado: Tá, meu nome completo é gran[dʒɪ], é Mateus Felipe Galera [dʒɪ] Oliveira Costa, eu faço [dʒɪ]reito e eu tô no quinto ano, úl[tʃɪ]mo período, no décimo. 
Entrevistadora: Perfeito. E o que que cê acha do curso? 
Entrevistado: Eu gosto do cu[ɹ(n)]so, eu acho que era realmen[tʃɪ] o que eu queria faze[∅r(v)], só que… eu posso fala[∅r(v)] sobre o cu[ɹ(n)]so da UEL mesmo, ou o cu[ɹ(n)]so geral, ah, okay, eu acho que poderia se[ɹ(v)] melho[ɹ(n)], inclusive eu tava conve[ɹ(v)]sa[n∅dʊ] hoje an[tɪ]s [dɪ] eu entra[∅r(v)] na sala isso, que poderia se[∅r(v)] melho[ɹ(n)], mas é um, é um bom cu[ɹ(n)]so, vale, vale a pena se[∅r(v)] feito sim.
Entrevistadora: E quais são as suas expectativas em relação ao curso agora que você está acabando? 
Entrevistado: Expecta[tʃɪ]va[s]… é [dʒɪ]

In [7]:
def df_find_cases_without_duplication(file_path):
    words_for_table = []  # Use a list to store repeated entries
    text = read_docx(file_path)
    # Use regular expression to find words with 't' inside brackets but without 'ʃ' right after it
    
    # padroes de patalizações
    pattern = r'\S*\[[^\]]*(?:tʃ|t(?!ʃ)|dʒ|de|dɪ)[^\]]*\]\S*'

    
    matches = re.findall(pattern, text)
    

    words_for_table.extend(matches)
    
    
    df = DataFrame(words_for_table, columns=['palavra'])
    
    #remove commas and periods at the end of values
    punctuation_pattern = r'[.,…!?]+$'
    a_pattern = r'-a'
    o_pattern = r'-o'
    pattern = punctuation_pattern + '|' + a_pattern + '|' + o_pattern
    df['palavra'] = df['palavra'].str.replace(pattern, '', regex=True)
        
            
    return df

file_path = "../data/subject1.docx"
df = df_find_cases_without_duplication(file_path)
df

,palavra
0,gran[dʒɪ]
1,[dʒɪ]
2,[dʒɪ]reito
3,úl[tʃɪ]mo
4,realmen[tʃɪ]
...,...
172,es[tɪ]ve[ɹ(v)][de][s]
173,[de]
174,[dɪ]ze[ɹ(v)]
175,fo[ɹ(n)][te][s]


In [8]:
def create_patalizacao(row):
    
    pattern_t = r'\[[^\]]*t(?!ʃ)[^\]]*\]'
    pattern_tf = r'\[[^\]]*tʃ[^\]]*\]'
    pattern_dz = r'\[[^\]]*dʒ[^\]]*\]'
    pattern_de = r'\[[^\]]*de[^\]]*\]'
    pattern_di = r'\[[^\]]*dɪ[^\]]*\]'

    pattern = pattern_t + '|' + pattern_tf + '|' + pattern_dz + '|' + pattern_de + '|' + pattern_di
    

    patalizacao_cru = re.findall(pattern, row['palavra'])
    
    

    if re.search(pattern_t, patalizacao_cru[0]):
        patalização = '[t]'
    if re.search(pattern_tf, patalizacao_cru[0]):
        patalização ='[tʃ]'
    if re.search(pattern_dz, patalizacao_cru[0]):
        patalização = '[dʒ]'
    if re.search(pattern_de, patalizacao_cru[0]):
        patalização = '[de]'
    if re.search(pattern_di, patalizacao_cru[0]):
        patalização = '[dɪ]'

    return patalizacao_cru, patalização

In [9]:
# Function to determine 'sonoridade' based on 'patalização' values
def determine_sonoridade(row):
    
    if row == '[t]' or row == '[tʃ]':
        return 'surda'
    elif row == '[de]' or row == '[dɪ]' or row == '[dʒ]':
        return 'sonora'
    
    return None


In [10]:

def create_vogais_alvo(row):
    vogais_alvo = []
    
    pattern_I = r'\[[^\]]*ɪ[^\]]*\]'
    pattern_e = r'\[[^\]]*e[^\]]*\]'
    pattern = pattern_I + '|' + pattern_e

    match = re.search(pattern, row['palavra'])

    if match:
        vogais_cru = match.group(0)

        if re.search(pattern_I, vogais_cru):
            return '[ɪ]'
        if re.search(pattern_e, vogais_cru):
            return '[e]'
        
    return None
    
    

In [11]:
def apply_cols(file_path):
    df = df_find_cases_without_duplication(file_path)
    df['patalizacao_cru'], df['patalização'] = zip(*df.apply(create_patalizacao, axis=1))
    
    # df['patalização_count'] = df['patalização'].apply(lambda x: len([item.strip() for item in x.split(',')]))
    df['sonoridade'] = df['patalização'].apply(determine_sonoridade)
    df['vogais_alvo'] = df.apply(create_vogais_alvo, axis=1)
    return df

In [12]:
df = apply_cols(file_path)
df

,palavra,patalizacao_cru,patalização,sonoridade,vogais_alvo
0,gran[dʒɪ],[[dʒɪ]],[dʒ],sonora,[ɪ]
1,[dʒɪ],[[dʒɪ]],[dʒ],sonora,[ɪ]
2,[dʒɪ]reito,[[dʒɪ]],[dʒ],sonora,[ɪ]
3,úl[tʃɪ]mo,[[tʃɪ]],[tʃ],surda,[ɪ]
4,realmen[tʃɪ],[[tʃɪ]],[tʃ],surda,[ɪ]
...,...,...,...,...,...
172,es[tɪ]ve[ɹ(v)][de][s],"[[tɪ], [de]]",[t],surda,[ɪ]
173,[de],[[de]],[de],sonora,[e]
174,[dɪ]ze[ɹ(v)],[[dɪ]],[dɪ],sonora,[ɪ]
175,fo[ɹ(n)][te][s],[[te]],[t],surda,[e]


In [13]:
def create_posicao_patalizacao_final(row):
    
    
    unico_pattern = r'^\[[^\]]*(t(?!ʃ)|tʃ|dʒ|de|dɪ)[[^\]]*\]$'

  
    comeco_pattern = r'^\[[^\]]*(t(?!ʃ)|tʃ|dʒ|de|dɪ)[^\]]*\]'

   
    meio_pattern = r'\[[^\]]*(tʃ|t(?!ʃ)|dʒ|de|dɪ)[^\]]*\]'
    meio_pattern_mais = r'\S\[[^\]]*(?:tʃ|t(?!ʃ)|dʒ|de|dɪ)[^\]]*\]\S'


    final_pattern = r'\[[^\]]*(t(?!ʃ)|tʃ|dʒ|de|dɪ)[^\]]*\]$'


    

    row['copy_palavra'] = row['palavra']

    if re.search(unico_pattern, row['palavra']):
        row['posição_patalização'] = row['patalização'] + 'único'
        row['copy_palavra'] = re.sub(unico_pattern, '', row['copy_palavra'])

    elif re.search(comeco_pattern, row['palavra']):
        row['posição_patalização'] = row['patalização'] + ' comeco'
        row['copy_palavra'] = re.sub(comeco_pattern, '', row['copy_palavra'])

    elif re.search(meio_pattern_mais, row['palavra']):
        row['posição_patalização'] = row['patalização'] + ' meio'
        row['copy_palavra'] = re.sub(meio_pattern, '', row['copy_palavra'], count=1)
    
    elif re.search(final_pattern, row['palavra']):
        row['posição_patalização'] = row['patalização'] + ' final'
        row['copy_palavra'] = re.sub(final_pattern, '', row['copy_palavra'])
    
    else:
        row['posição_patalização'] = ''
    return row


df = df.apply(create_posicao_patalizacao_final, axis=1)
df


/tmp/ipykernel_7086/3712872512.py:21: FutureWarning: Possible nested set at position 30
  if re.search(unico_pattern, row['palavra']):


,palavra,patalizacao_cru,patalização,sonoridade,vogais_alvo,copy_palavra,posição_patalização
0,gran[dʒɪ],[[dʒɪ]],[dʒ],sonora,[ɪ],gran,[dʒ] final
1,[dʒɪ],[[dʒɪ]],[dʒ],sonora,[ɪ],,[dʒ] comeco
2,[dʒɪ]reito,[[dʒɪ]],[dʒ],sonora,[ɪ],reito,[dʒ] comeco
3,úl[tʃɪ]mo,[[tʃɪ]],[tʃ],surda,[ɪ],úlmo,[tʃ] meio
4,realmen[tʃɪ],[[tʃɪ]],[tʃ],surda,[ɪ],realmen,[tʃ] final
...,...,...,...,...,...,...,...
172,es[tɪ]ve[ɹ(v)][de][s],"[[tɪ], [de]]",[t],surda,[ɪ],esve[ɹ(v)][de][s],[t] meio
173,[de],[[de]],[de],sonora,[e],,[de]único
174,[dɪ]ze[ɹ(v)],[[dɪ]],[dɪ],sonora,[ɪ],ze[ɹ(v)],[dɪ] comeco
175,fo[ɹ(n)][te][s],[[te]],[t],surda,[e],fo[ɹ(n)][s],[t] meio


In [17]:
pd.set_option('display.max_rows', None)

In [18]:
df

,palavra,patalizacao_cru,patalização,sonoridade,vogais_alvo,copy_palavra,posição_patalização
0,gran[dʒɪ],[[dʒɪ]],[dʒ],sonora,[ɪ],gran,[dʒ] final
1,[dʒɪ],[[dʒɪ]],[dʒ],sonora,[ɪ],,[dʒ] comeco
2,[dʒɪ]reito,[[dʒɪ]],[dʒ],sonora,[ɪ],reito,[dʒ] comeco
3,úl[tʃɪ]mo,[[tʃɪ]],[tʃ],surda,[ɪ],úlmo,[tʃ] meio
4,realmen[tʃɪ],[[tʃɪ]],[tʃ],surda,[ɪ],realmen,[tʃ] final
5,an[tɪ]s,[[tɪ]],[t],surda,[ɪ],ans,[t] meio
6,[dɪ],[[dɪ]],[dɪ],sonora,[ɪ],,[dɪ]único
7,Expecta[tʃɪ]va[s],[[tʃɪ]],[tʃ],surda,[ɪ],Expectava[s],[tʃ] meio
8,[dʒɪ]sso,[[dʒɪ]],[dʒ],sonora,[ɪ],sso,[dʒ] comeco
9,basicamen[tʃɪ],[[tʃɪ]],[tʃ],surda,[ɪ],basicamen,[tʃ] final


In [22]:
df = apply_cols(file_path)
df = df.apply(create_posicao_patalizacao_final, axis=1)

In [23]:
def df_add_duplicated_matches(file_path):
    df = apply_cols(file_path)
    df = df.apply(create_posicao_patalizacao_final, axis=1)

    # Regular expression pattern
    pattern = r'\[[^\]]*(t(?!ʃ)|tʃ|dʒ|de|dɪ)[^\]]*\]'

    
    # Find matches for each value in the column
    matches = df['palavra'].str.findall(pattern)
    
    # idenfitfy idexes where there is a match
    match_index = df[matches.apply(lambda x: len(x) > 1)].index
    
    #turn index object to list
    match_list = list(match_index)
    
    i = 0
    for element in match_list:

        element_used = element + i
        new_row_data = df.loc[element_used:element_used]
        new_row_data = new_row_data[['palavra','copy_palavra', 'patalizacao_cru']]

        df = concat([df.iloc[:element_used + 1], new_row_data, df.iloc[element_used + 1:]]).reset_index(drop=True)
        i = i + 1
    
    df = df.fillna('') 
    return df
# pd.set_option('display.maxs_rows', None)
df = df_add_duplicated_matches(file_path)


In [24]:
def new_create_patalizacao(row, i):
    
    pattern_t = r'\[[^\]]*t(?!ʃ)[^\]]*\]'
    pattern_tf = r'\[[^\]]*tʃ[^\]]*\]'
    pattern_dz = r'\[[^\]]*dʒ[^\]]*\]'
    pattern_de = r'\[[^\]]*de[^\]]*\]'
    pattern_di = r'\[[^\]]*dɪ[^\]]*\]'

    pattern = pattern_t + '|' + pattern_tf + '|' + pattern_dz + '|' + pattern_de + '|' + pattern_di
    
    
    patalizacao_cru = re.findall(pattern, row['palavra'])

    if row['patalização'] == '':
        if re.search(pattern_t, patalizacao_cru[i]):
            return '[t]'
        if re.search(pattern_tf, patalizacao_cru[i]):
            return '[tʃ]'
        if re.search(pattern_dz, patalizacao_cru[i]):
            return '[dʒ]'
        if re.search(pattern_de, patalizacao_cru[i]):
            return '[de]'
        if re.search(pattern_di, patalizacao_cru[i]):
            return '[dɪ]'

    return row['patalização']

# df['patalização'] = df.apply(new_create_patalizacao, axis=1)
df['patalização'] = df.apply(lambda row: new_create_patalizacao(row, 1), axis=1)

In [25]:
df['sonoridade'] = df['patalização'].apply(determine_sonoridade)


In [63]:
def new_create_vogais_alvo(row):
    vogais_alvo = []
    
    pattern_I = r'\[[^\]]*ɪ[^\]]*\]'
    pattern_e = r'\[[^\]]*e[^\]]*\]'

    patalizacao_cru = row['patalizacao_cru']

    if row['vogais_alvo'] == '':
        if re.search(pattern_I, patalizacao_cru[1]):
            return '[ɪ]'
        if re.search(pattern_e, patalizacao_cru[1]):
            return '[e]'
        
    return row['vogais_alvo']
    

In [64]:
df['vogais_alvo'] = df.apply(create_vogais_alvo, axis=1)

In [26]:
def new_create_posicao_patalizacao_final(row):
    
    
    unico_pattern = r'^\[[^\]]*(t(?!ʃ)|tʃ|dʒ|de|dɪ)[[^\]]*\]$'

  
    comeco_pattern = r'^\[[^\]]*(t(?!ʃ)|tʃ|dʒ|de|dɪ)[^\]]*\]'

   
    meio_pattern = r'\[[^\]]*(tʃ|t(?!ʃ)|dʒ|de|dɪ)[^\]]*\]'
    meio_pattern_mais = r'\S\[[^\]]*(?:tʃ|t(?!ʃ)|dʒ|de|dɪ)[^\]]*\]\S'


    final_pattern = r'\[[^\]]*(t(?!ʃ)|tʃ|dʒ|de|dɪ)[^\]]*\]$'


    

    if row['posição_patalização'] == '':


        if re.search(comeco_pattern, row['copy_palavra']) or re.search(meio_pattern_mais, row['copy_palavra']):
            row['posição_patalização'] = row['patalização'] + ' meio'

        elif re.search(final_pattern, row['copy_palavra']):
            row['posição_patalização'] = row['patalização'] + ' final'
    return row


df = df.apply(new_create_posicao_patalizacao_final, axis=1)


In [27]:
df.drop(columns=['patalizacao_cru', 'copy_palavra'])

,palavra,patalização,sonoridade,vogais_alvo,posição_patalização
0,gran[dʒɪ],[dʒ],sonora,[ɪ],[dʒ] final
1,[dʒɪ],[dʒ],sonora,[ɪ],[dʒ] comeco
2,[dʒɪ]reito,[dʒ],sonora,[ɪ],[dʒ] comeco
3,úl[tʃɪ]mo,[tʃ],surda,[ɪ],[tʃ] meio
4,realmen[tʃɪ],[tʃ],surda,[ɪ],[tʃ] final
5,an[tɪ]s,[t],surda,[ɪ],[t] meio
6,[dɪ],[dɪ],sonora,[ɪ],[dɪ]único
7,Expecta[tʃɪ]va[s],[tʃ],surda,[ɪ],[tʃ] meio
8,[dʒɪ]sso,[dʒ],sonora,[ɪ],[dʒ] comeco
9,basicamen[tʃɪ],[tʃ],surda,[ɪ],[tʃ] final
